# Setup

In [19]:
from pathlib import Path
import sys

section_dir = Path.cwd()
sys.path.append(str(section_dir))
print(section_dir.name)

OTHELLO_ROOT = (section_dir / "othello_world").resolve()
OTHELLO_MECHINT_ROOT = (OTHELLO_ROOT / "mechanistic_interpretability").resolve()

# if not OTHELLO_ROOT.exists():
#     !git clone https://github.com/likenneth/othello_world

sys.path.append(str(OTHELLO_MECHINT_ROOT))

from mech_interp_othello_utils import (
    plot_board,
    plot_single_board,
    plot_board_log_probs,
    to_string,
    to_int,
    int_to_label,
    string_to_label,
    OthelloBoardState
)

# import deepcopy
from copy import deepcopy

interpretability


# The Tactic
- Two Games
- Play the same random stones in the two games
- Before each, search for a way to place two stones such that the stone has a different color... in the corrupt patch
- In the Clean Patch do a random move
- After that play the same move as long as possible

In [21]:
board = OthelloBoardState()
# board.get_state()
# board.initial_state
print(board.state)
board.umpire(19)
print(board.state)
print(board.get_valid_moves())
board.history

history = deepcopy(board.history)

[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
[18, 20, 34]


In [23]:
import random

# This function test all possibilities of two moves to see if they can play move_clean on the second move
def get_corrupt_move(board_corrupt, move_clean):
    # board_state_initial = board_corrupt.state.copy()
    valid_moves_corrupt_initial = board_corrupt.get_valid_moves().copy()
    for move1 in valid_moves_corrupt_initial:
        board_corrupt_copy = deepcopy(board_corrupt)
        # board_corrupt_copy = board_corrupt.copy()
        board_corrupt_copy.umpire(move1)
        valid_moves_corrupt = board_corrupt_copy.get_valid_moves().copy()
        # board_corrupt.state = board_state_initial
        if move_clean in valid_moves_corrupt:
            return move1
    return move_clean

def calculate_next_board(board_clean, board_corrupt):
    valid_moves_clean = board_clean.get_valid_moves()
    move_clean = random.choice(valid_moves_clean)
    move_corrupt = get_corrupt_move(board_corrupt, move_clean)
    board_clean.umpire(move_clean)
    board_corrupt.umpire(move_corrupt)
    if move_clean == move_corrupt:
        did_corruption_occur = False
        return did_corruption_occur
    valid_moves_clean = board_clean.get_valid_moves()
    move_clean_new = random.choice(valid_moves_clean)
    move_corrupt_new = move_clean
    board_clean.umpire(move_clean_new)
    board_corrupt.umpire(move_corrupt_new)
    did_corruption_occur = True
    return did_corruption_occur

def play_same_move(board_clean, board_corrupt):
    valid_moves_clean = board_clean.get_valid_moves()
    valid_moves_corrupt = board_corrupt.get_valid_moves()
    valid_moves_both = list(set(valid_moves_clean) & set(valid_moves_corrupt))
    if len(valid_moves_both) == 0:
        was_move_possible = False
        return was_move_possible
    move = random.choice(valid_moves_both)
    board_clean.umpire(move)
    board_corrupt.umpire(move)
    was_move_possible = True
    return was_move_possible
    
def get_patch():
    move = 0
    board_clean = OthelloBoardState()
    board_corrupt = OthelloBoardState()
    did_corruption_occur = False
    while not did_corruption_occur:
        did_corruption_occur = calculate_next_board(board_clean, board_corrupt)
        if did_corruption_occur:
            move += 2
        else:
            move += 1
    corrupted_square = board_corrupt.history[-1]
    corrupted_move = move-1
    was_move_possible = True
    while was_move_possible:
        was_move_possible = play_same_move(board_clean, board_corrupt)
        move += 1
    assert len(board_clean.history) == len(board_corrupt.history)
    length = len(board_clean.history)
    return board_clean.history, board_corrupt.history, corrupted_square, corrupted_move, length

for i in range(5):
    board_history_clean, board_history_corrupt, corrupted_square, corrupted_move, length = get_patch()
    if length >= 20:
        print(f"board_history_clean:   {board_history_clean}")
        print(f"board_history_corrupt: {board_history_corrupt}")
        print(f"corrupted_square: {corrupted_square}")
        print(f"corrupted_move: {corrupted_move}")
        print(f"length: {length}")

board_history_clean:   [44, 29, 21, 45, 37, 43, 53, 22, 18, 38, 30, 9, 47, 14, 42, 49, 20, 13, 26, 34, 25, 41, 7, 32, 12, 17, 0, 19, 48, 46, 33, 24, 10, 6, 50, 60, 16, 11, 2, 56, 40, 52, 57, 8, 61, 62, 4, 59, 5, 58, 55, 54, 63, 1, 3, 39, 31, 15, 23]
board_history_corrupt: [44, 29, 21, 45, 37, 43, 51, 53, 18, 38, 30, 9, 47, 14, 42, 49, 20, 13, 26, 34, 25, 41, 7, 32, 12, 17, 0, 19, 48, 46, 33, 24, 10, 6, 50, 60, 16, 11, 2, 56, 40, 52, 57, 8, 61, 62, 4, 59, 5, 58, 55, 54, 63, 1, 3, 39, 31, 15, 23]
corrupted_square: 53
corrupted_move: 7
length: 59
board_history_clean:   [44, 43, 34, 25, 26, 19, 33, 37, 42, 52, 38, 18, 53, 54, 11, 29, 21, 39, 45, 4, 61, 50, 55, 22, 24, 63, 60, 51, 30, 47, 15, 32, 41, 16, 57, 40, 58, 59, 46, 62, 17, 8, 49, 12, 10, 23, 14, 1, 48, 13, 2, 5, 7, 9, 3, 6, 0, 56, 31]
board_history_corrupt: [44, 43, 34, 25, 26, 20, 19, 37, 42, 52, 38, 18, 53, 54, 11, 29, 21, 39, 45, 4, 61, 50, 55, 22, 24, 63, 60, 51, 30, 47, 15, 32, 41, 16, 57, 40, 58, 59, 46, 62, 17, 8, 49, 12, 10